# Longitudinal Data Visualization Project

In [ ]:
!pip install -r requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

In [ ]:
import google.cloud

In [ ]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [ ]:
credentials = service_account.Credentials.from_service_account_file(
    "./getusppe-93fe2669c4a3.json",
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id,
)
QUERY = ('select * from gateway.request limit 10')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

In [ ]:
query_string = "SELECT * FROM gateway.request"
bq_df = client.query(query_string).to_dataframe()

In [ ]:
pd.set_option('display.max_columns', None)
display(bq_df)

In [ ]:
# Filename parameters
requestors_filename = "requestor-Grid view.csv"
requestors_timestamped_filename = "data-1591226924897.csv"
output_new_timestamps_filename = "requestors_new_timestamps.csv"
output_sorted_binary_filename = "requestors_new_timestamps_sorted_binary.csv"

In [ ]:
# Read csv file with requestors
reqs = pd.io.parsers.read_csv(requestors_filename, low_memory=False)
reqs_ts = pd.io.parsers.read_csv(requestors_timestamped_filename)

In [ ]:
# Match new timestamps from Airtable database with GCP BigQuery, append to list
new_timestamps = []
for index, row in reqs.iterrows():
    # Filter out by state/city first, then run extract
    df = bq_df[(bq_df['requestkey'] == row['requestKey'])]
    if len(df.index) == 1:
        new_timestamps.append(df.iloc[0]['datecreated'])
    else:
        new_timestamps.append('N/A')

In [ ]:
# # Match new timestamps from Airtable database, append to list
# new_timestamps = []
# for index, row in reqs.iterrows():
#     # Filter out by state/city first, then run extract
#     df = reqs_ts[(reqs_ts['requestkey'] == row['requestKey'])]
#     if len(df.index) == 1:
#         new_timestamps.append(df.iloc[0]['datecreated'])
#     else:
#         new_timestamps.append('N/A')

In [ ]:
# Create new column for the fixed timestamps in the original database
reqs['newDateCreated'] = new_timestamps

In [ ]:
reqs

In [ ]:
# Export with new timestamps
reqs.to_csv(output_new_timestamps_filename, index=False)

In [ ]:
# Convert to datetime objects
reqs['newDateCreated'] = pd.to_datetime(reqs['newDateCreated'], errors='coerce', format="%Y-%m-%d %H:%M:%S", utc=True)

# Sort requests by date, ascending
sorted_reqs = reqs.sort_values(by='newDateCreated')

In [ ]:
# Reset index
sorted_reqs = sorted_reqs.reset_index(drop=True)

In [ ]:
sorted_reqs

In [ ]:
# Create binary yes/no columns for requests by type (>0 requested, or == -1 (binary yes/no))
sorted_reqs['requestedRespirators'] = sorted_reqs.apply(lambda row : row['respirators'] > 0 or row['respirators'] < 0, axis=1)
sorted_reqs['requestedSurgicalMasks'] = sorted_reqs.apply(lambda row : row['surgicalMasks'] > 0 or row['surgicalMasks'] < 0, axis=1)
sorted_reqs['requestedFaceShields'] = sorted_reqs.apply(lambda row : row['faceShields'] > 0 or row['faceShields'] < 0, axis=1)
sorted_reqs['requestedSafetyGoggles'] = sorted_reqs.apply(lambda row : row['safetyGoggles'] > 0 or row['safetyGoggles'] < 0, axis=1)
sorted_reqs['requestedSafetyGlasses'] = sorted_reqs.apply(lambda row : row['safetyGlasses'] > 0 or row['safetyGlasses'] < 0, axis=1)
sorted_reqs['requestedDisposableBooties'] = sorted_reqs.apply(lambda row : row['disposableBooties'] > 0 or row['disposableBooties'] < 0, axis=1)
sorted_reqs['requestedNitrileGloves'] = sorted_reqs.apply(lambda row : row['nitrileGloves'] > 0 or row['nitrileGloves'] < 0, axis=1)
sorted_reqs['requestedGowns'] = sorted_reqs.apply(lambda row : row['gowns'] > 0 or row['gowns'] > 0, axis=1)
sorted_reqs['requestedCoveralls'] = sorted_reqs.apply(lambda row : row['coveralls'] > 0 or row['coveralls'] < 0, axis=1)
sorted_reqs['requestedSurgicalCaps'] = sorted_reqs.apply(lambda row : row['surgicalCaps'] > 0 or row['surgicalCaps'] < 0, axis=1)
sorted_reqs['requestedHandSanitizer'] = sorted_reqs.apply(lambda row : row['handSanitizer'] > 0 or row['handSanitizer'] < 0, axis=1)
sorted_reqs['requestedDisinfectingWipes'] = sorted_reqs.apply(lambda row : row['disinfectingWipes'] > 0 or row['disinfectingWipes'] < 0, axis=1)
sorted_reqs['requestedThermometers'] = sorted_reqs.apply(lambda row : row['thermometers'] > 0 or row['thermometers'] < 0, axis=1)
sorted_reqs['requestedHandmadeMasks'] = sorted_reqs.apply(lambda row : row['handmadeMasks'] > 0 or row['handmadeMasks'] < 0, axis=1)
sorted_reqs['requestedPrintedFaceShields'] = sorted_reqs.apply(lambda row : row['printedFaceShields'] > 0 or row['printedFaceShields'] < 0, axis=1)
sorted_reqs['requestedGownsOrCoveralls'] = sorted_reqs.apply(lambda row : row['gowns'] > 0 or row['gowns'] > 0 or row['coveralls'] > 0 or row['coveralls'] < 0, axis=1)

In [ ]:
# Save to csv file
sorted_reqs.to_csv(output_sorted_binary_filename, index=False)

In [ ]:
# Start and end date from dataset
start_date = min(sorted_reqs['newDateCreated'].dt.date)
end_date = max(sorted_reqs['newDateCreated'].dt.date)

In [ ]:
# Create timeseries slicing by day
idx = pd.date_range(start_date, periods=(end_date - start_date).days + 1, freq='D')
df_sliced_day = idx.to_frame(index=False, name='date')

In [ ]:
# Create columns for each type of PPE
types = ['respirators', 'surgicalMasks', 'faceShields', 'safetyGoggles', 'safetyGlasses', 'disposableBooties', 'nitrileGloves', 'gowns', 'coveralls', 'surgicalCaps', 'handSanitizer', 'disinfectingWipes', 'thermometers', 'handmadeMasks', 'printedFaceShields', 'gownsOrCoveralls']
for t in types:
    df_sliced_day[t] = 0.0

In [ ]:
# Using this method to convert from timestamp -> datetime object
idx[0].to_pydatetime()

In [ ]:
# # Tally up totals
# types_reqs = ['requested' + t[0].upper() + t[1:] for t in types]
# for index, row in df_sliced_day.iterrows():
#     for i in range(len(types)):
#         # all matching dates
#         total = sorted_reqs[(sorted_reqs['newDateCreated'].dt.normalize() == row['date'].to_pydatetime())]
#         # all matching dates and item was requested is True
#         items = sorted_reqs[(sorted_reqs['newDateCreated'].dt.normalize() == row['date'].to_pydatetime()) & (sorted_reqs[types_reqs[i]])]
# #         print(temp)

#         # Normalize based on each date's total number of requests
#         if len(total.index) == 0:  # don't divide by 0
#             df_sliced_day.at[index, types[i]] = 0
#         else:
#             df_sliced_day.at[index, types[i]] = len(total.index) 

In [ ]:
# Remove timezone to avoid error comparing naive datetime objects to timezone-aware ones
sorted_reqs['newDateCreated'] = sorted_reqs['newDateCreated'].dt.tz_convert(None)

In [ ]:
# Tally up binary yes's/no's for each day
types_reqs = ['requested' + t[0].upper() + t[1:] for t in types]
for index, row in df_sliced_day.iterrows():
    for i in range(len(types)):
        # all matching dates
        total = sorted_reqs[(sorted_reqs['newDateCreated'].dt.normalize() == row['date'].to_pydatetime())]
        # all matching dates and item was requested is True
        items = sorted_reqs[(sorted_reqs['newDateCreated'].dt.normalize() == row['date'].to_pydatetime()) & (sorted_reqs[types_reqs[i]])]
#         print(temp)

        # Normalize based on each date's total number of requests
        if len(total.index) == 0:  # don't divide by 0
            df_sliced_day.at[index, types[i]] = 0
        else:
            df_sliced_day.at[index, types[i]] = len(items.index) * 1.0 / len(total.index) 

In [ ]:
df_sliced_day

In [ ]:
dates = [x.date() for x in list(df_sliced_day['date'])]
dates_as_nums = [x.toordinal() for x in dates]

In [ ]:
# Total number of requests by day
# plt.clf()

# dates = [x.date() for x in list(df_sliced_day['date'])]
# dates_as_nums = [x.toordinal() for x in dates]

# fig, ax = plt.subplots(figsize=(15,15))
# for t in ['respirators']:
#     coef = np.polyfit(dates_as_nums,df_sliced_day[t],1)
#     poly1d_fn = np.poly1d(coef)
#     ax.plot(dates, df_sliced_day[t])
#     ax.plot(dates_as_nums, poly1d_fn(dates_as_nums), label="Total number of requests")
# ax.legend()

# # ax.set_xticks(np.arange(len(dates)))
# # ax.set_xticklabels(dates)
# plt.title('Total PPE Requests, Sliced by Day')
# plt.xlabel('Date')
# plt.ylabel('Number of Requests')
# plt.show()

In [ ]:
# Polynomial Regression
# From: https://stackoverflow.com/questions/893657/how-do-i-calculate-r-squared-using-python-and-numpy
def polyfit(x, y, degree):
    results = {}

    coeffs = np.polyfit(x, y, degree)

     # Polynomial Coefficients
    results['polynomial'] = coeffs.tolist()

    # r-squared
    p = np.poly1d(coeffs)
    # fit values, and mean
    yhat = p(x)                         # or [p(z) for z in x]
    ybar = np.sum(y)/len(y)          # or sum(y)/len(y)
    ssreg = np.sum((yhat-ybar)**2)   # or sum([ (yihat - ybar)**2 for yihat in yhat])
    sstot = np.sum((y - ybar)**2)    # or sum([ (yi - ybar)**2 for yi in y])
    results['rsquared'] = ssreg / sstot

    return results

In [ ]:
# Plot sliced by day

plt.clf()

dates = [x.date() for x in list(df_sliced_day['date'])]
dates_as_nums = [x.toordinal() for x in dates]

fig, ax = plt.subplots(figsize=(20,20))
for t in types:
    results = polyfit(dates_as_nums,df_sliced_day[t],1)
    coef = results['polynomial']
    poly1d_fn = np.poly1d(coef)
    ax.scatter(dates, df_sliced_day[t], label=t)
    ax.plot(dates_as_nums, poly1d_fn(dates_as_nums), label=(t + " | R^2 = " + str(round(results['rsquared'],3))))
ax.legend()

# ax.set_xticks(np.arange(len(dates)))
# ax.set_xticklabels(dates)
plt.title('Type of PPE Requested Over Time, Sliced by Day')
plt.xlabel('Date')
plt.ylabel('Normalized Number of Requests (Fraction of Requests Per Day By Type)')
plt.show()

In [ ]:
# Plot only respirators, face shields, and gowns
plt.clf()

dates = [x.date() for x in list(df_sliced_day['date'])]
dates_as_nums = [x.toordinal() for x in dates]

fig, ax = plt.subplots(figsize=(15,15))
for t in ['respirators', 'faceShields', 'gownsOrCoveralls']:
    coef = np.polyfit(dates_as_nums,df_sliced_day[t],1)
    poly1d_fn = np.poly1d(coef)
    ax.scatter(dates, df_sliced_day[t], label=t)
    ax.plot(dates_as_nums, poly1d_fn(dates_as_nums), label=t)
ax.legend()

# ax.set_xticks(np.arange(len(dates)))
# ax.set_xticklabels(dates)
plt.title('Type of PPE Requested Over Time, Sliced by Day')
plt.xlabel('Date')
plt.ylabel('Normalized Number of Requests (Fraction of Requests Per Day By Type)')
plt.show()

In [ ]:
# Plot each type on own individual plot
plt.clf()

dates = [x.date() for x in list(df_sliced_day['date'])]
dates_as_nums = [x.toordinal() for x in dates]


for t in types:
    fig, ax = plt.subplots(figsize=(10,10))
    coef = np.polyfit(dates_as_nums,df_sliced_day[t],1)
    poly1d_fn = np.poly1d(coef)
    ax.scatter(dates, df_sliced_day[t], label=t)
    ax.plot(dates_as_nums, poly1d_fn(dates_as_nums), label=t)
    ax.legend()
    plt.title('Type of PPE Requested Over Time, Sliced by Day')
    plt.xlabel('Date')
    plt.ylabel('Normalized Number of Requests (Fraction of Requests Per Day By Type)')
    plt.show()


In [ ]:
# Create timeseries slicing by week
idx = pd.date_range(start_date, periods=((end_date - start_date).days + 1) // 7 , freq='7D')
df_sliced_week = idx.to_frame(index=False, name='date')

In [ ]:
# Create columns for each type of PPE
types = ['respirators', 'surgicalMasks', 'faceShields', 'safetyGoggles', 'safetyGlasses', 'disposableBooties', 'nitrileGloves', 'gowns', 'coveralls', 'surgicalCaps', 'handSanitizer', 'disinfectingWipes', 'thermometers', 'handmadeMasks', 'printedFaceShields']
for t in types:
    df_sliced_week[t] = 0.0

In [ ]:
# Tally up binary yes's/no's for each week
types_reqs = ['requested' + t[0].upper() + t[1:] for t in types]
for index, row in df_sliced_week.iterrows():
    for i in range(len(types)):
        # all matching dates
        total = sorted_reqs[(sorted_reqs['newDateCreated'].dt.normalize() >= row['date'].to_pydatetime()) & (sorted_reqs['newDateCreated'].dt.normalize() < row['date'].to_pydatetime() + timedelta(days=7))]
        # all matching dates and item was requested is True
        items = sorted_reqs[(sorted_reqs['newDateCreated'].dt.normalize() == row['date'].to_pydatetime()) & (sorted_reqs['newDateCreated'].dt.normalize() < row['date'].to_pydatetime() + timedelta(days=7)) & (sorted_reqs[types_reqs[i]])]
#         print(temp)

        # Normalize based on each date's total number of requests
        if len(total.index) == 0:  # don't divide by 0
            df_sliced_week.at[index, types[i]] = 0
        else:
            df_sliced_week.at[index, types[i]] = len(items.index) * 1.0 / len(total.index) 

In [ ]:
df_sliced_week

In [ ]:
plt.clf()

dates = [x.date() for x in list(df_sliced_week['date'])]
dates_as_nums = [x.toordinal() for x in dates]

fig, ax = plt.subplots(figsize=(15,15))
for t in types:
    coef = np.polyfit(dates_as_nums,df_sliced_week[t],1)
    poly1d_fn = np.poly1d(coef)
    ax.scatter(dates, df_sliced_week[t], label=t)
    ax.plot(dates_as_nums, poly1d_fn(dates_as_nums), label=t)
ax.legend()

# ax.set_xticks(np.arange(len(dates)))
# ax.set_xticklabels(dates)
plt.title('Type of PPE Requested Over Time, Sliced by Week')
plt.xlabel('Week Of')
plt.ylabel('Normalized Number of Requests (Fraction of Requests Per Week By Type)')
plt.show()